In [1]:
import os
import re

import pandas as pd

In [2]:
sampled_articles = pd.read_csv("../data/sampled_articles_details.csv")

lexicon = pd.read_csv("Sensitivity/norm_lexicon__5.csv")
top_terms_articles = pd.read_csv("top_terms_by_mean_weight__articles.csv", index_col=[0])

C:\Users\mattb\Miniconda3\envs\peace\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
lexicon_articles = lexicon.merge(top_terms_articles, on="term", how="left")

possible_articles = lexicon_articles.groupby("article_id") \
    .size() \
    .rename("n") \
    .reset_index() \
    .query("n >= 4")

filtered_terms = lexicon_articles[lexicon_articles.article_id.isin(possible_articles.article_id)]

In [4]:
anec_articles_ids = filtered_terms.groupby(["society", "article_id"]) \
    .size() \
    .rename("n") \
    .reset_index() \
    .groupby("society") \
    .sample(10, random_state=116) \
    .article_id \
    .to_list()

print(anec_articles_ids)

[1844601, 30977823, 1884282, 1793428, 40533898, 40881127, 30865248, 1866797, 14060426, 1815672, 1680309, 24167359, 3069080, 60626294, 40374416, 41990744, 42122954, 1172040, 2803082, 830141]


In [5]:
rnn_data = pd.read_csv("../data/domestic_articles__ngram__stopwords__lemmatized.csv", usecols=[1, 5])
anec_articles_rnn_text = rnn_data[rnn_data.article_id.isin(anec_articles_ids)].copy()

del rnn_data
anec_articles_rnn_text.head()

,article_id,article_text
46327,24167359,Motor race Gasly come back earth bump SHANGHAI...
76199,30865248,U 16 clinch one day series well The third fina...
76263,30977823,Fifth grader rap Kurigram Accused try cover Ku...
80125,60626294,As Ottawa free fall Belleville soar Senators p...
113936,830141,Terence Corcoran Canada 's Big Fat Chobani Gre...


In [6]:
anec_articles_top_terms = top_terms_articles[
    top_terms_articles.article_id.isin(anec_articles_ids)
] \
    .groupby("article_id") \
    .apply(lambda x: x.term.values) \
    .rename("top_terms") \
    .reset_index()

anec_articles_top_terms.head()

,article_id,top_terms
0,830141,"[please, produce, send, already, along]"
1,1172040,"[united, kingdom, follow, require, recently]"
2,1680309,"[except, certain, require, site, around]"
3,1793428,"[usually, grace, abroad, meant, also]"
4,1815672,"[embark, please, lose, according, leave]"


In [8]:
clean_sample_path = os.path.join("D:", "Peace Speech Project Drive", "clean_sample")
raw_sample_path = os.path.join("D:", "Peace Speech Project Drive", "sample_raw")

In [53]:
anec_article = sampled_articles[
    sampled_articles.article_id.isin(anec_articles_ids)
].copy()

anec_article = anec_article.merge(
    anec_articles_top_terms, on="article_id"
)

anec_article = anec_article.merge(
    anec_articles_rnn_text, on="article_id"
)

def get_title(p):
    with open(os.path.join(clean_sample_path, p)) as f:
        lines = f.readlines()
        f.close()
        
    return lines[1].strip().title()

def get_og_text(p):
    with open(os.path.join(raw_sample_path, p)) as f:
        lines = f.readlines()
        f.close()
        
    return lines[-1].strip()

anec_article["article_title"] = anec_article.path.apply(get_title)
anec_article["article_og_text"] = anec_article.path.apply(get_og_text)

peaceful_countries = ['GB', 'AU', 'CA', 'SG', 'NZ', 'IE']
non_peaceful_countries = ['PK', 'BD', 'NG', 'KE', 'ZA', 'TZ']

from typing import List
def get_society_label(country: str, peaceful: List[str], nonpeaceful: List[str]) -> str:
    if country in peaceful:
        return "peaceful"
    elif country in nonpeaceful:
        return "nonpeaceful"
    else:
        return "other"
    
anec_article["society"] = anec_article.country.apply(
    get_society_label, peaceful=peaceful_countries, nonpeaceful=non_peaceful_countries)

anec_article.head(1).T

,0
country,AU
publisher,sbs
year,2018
month,4
id,24167359
article_id,24167359
article_publisher,SBS
has_text,True
path,AU\sbs\2018\24167359_AU_15-04-18.txt
top_terms,"[subscribe, australia, follow, listen, around]"


In [10]:
peaceful_countries = ['GB', 'AU', 'CA', 'SG', 'NZ', 'IE']
non_peaceful_countries = ['PK', 'BD', 'NG', 'KE', 'ZA', 'TZ']

from typing import List
def get_society_label(country: str, peaceful: List[str], nonpeaceful: List[str]) -> str:
    if country in peaceful:
        return "peaceful"
    elif country in nonpeaceful:
        return "nonpeaceful"
    else:
        return "other"
    
anec_article["society"] = anec_article.country.apply(
    get_society_label, peaceful=peaceful_countries, nonpeaceful=non_peaceful_countries)

In [40]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 

from nltk.tokenize import word_tokenize

In [46]:
terms

array(['subscribe', 'australia', 'follow', 'listen', 'around'],
      dtype=object)

In [65]:
t = anec_article.iloc[0].article_og_text
terms = anec_article.iloc[0].top_terms

def pretty_og_text(t, terms):
    t = re.sub(r"<\w>", "", t).strip()
    t = re.sub(r" \s+", r" ", t)
    t = re.sub(r'\s. "', '."', t)
    t = re.sub(r' ,', ',', t)
    t = re.sub(r"\s(.?'\w)", r"\1", t)

    final_terms = []
    for t in word_tokenize(t):
        t_ = t.lower()
        if t_ in terms or lemmatizer.lemmatize(t_) in terms:
            final_terms.append('<span style="color: red">{:s}</span>'.format(t))
        else:
            final_terms.append(t)

    return " ".join(final_terms)

In [66]:
def pretty_cleaned_text(text, top_terms, format_type):
    pass

def pretty_rnn_text(text, top_terms, format_type):
    
    if format_type == "html":
        f_string = '<span style="color: red">{:s}</span>'
    elif format_type == "markdown":
        f_string = '**{:s}**'
    else:
        f_string = '{:s}'
        
    for term in top_terms:
        text = re.sub(r"\b{:s}\b".format(term), f_string.format(term), text)
        
    return text
    
def make_pretty_text(rnn_text, top_terms, format_type):
    pretty = pretty_rnn_text(rnn_text, top_terms, format_type)
    return pretty

def make_pretty_html(article_id, society, country, title, terms, text, format_type):
    if format_type == "html":
        f_string = '<div>Article #{:d}<br><h3>{:s} - {:s}</h3><strong>{:s}</strong><br><p>Top weighted terms: {:s}</p><p>{:s}</p></div>'
    
    return f_string.format(article_id, society.title(), country, title, terms, text) 
    
def apply_make_pretty(row, format_type="html"):
#     text = make_pretty_text(row.article_text.lower(), row.top_terms, format_type)
    text = pretty_og_text(row.article_og_text, row.top_terms)
    final_terms = []
    for term in sorted(row.top_terms):
        if term in lexicon.term.unique():
            final_terms.append("<strong>{:s}</strong>".format(term))
        else:
            final_terms.append(term)
            
    terms = ", ".join(final_terms)
    return make_pretty_html(row.article_id, row.society, row.country, row.article_title, terms, text, format_type)

anec_article["formatted"] = anec_article.apply(apply_make_pretty, axis=1)

In [67]:
formatted = anec_article.groupby("society").apply(lambda x: x.formatted.values)

In [68]:
with open("Peaceful RAW Article Sample.html", "w") as f:
    f.writelines("<hr>".join(formatted["peaceful"]))
    f.close()

with open("Non-Peaceful RAW Article Sample.html", "w") as f:
    f.writelines("<hr>".join(formatted["nonpeaceful"]))
    f.close()